# 1.Import thư viện và load dữ liệu


Bước đầu tiên trong quy trình thực nghiệm là nhập các thư viện Python chuyên dụng (`numpy`, `pandas`, `sklearn`) và nạp tập dữ liệu **Digits** (số viết tay) từ `sklearn.datasets`. Đây là bước chuẩn bị nguyên liệu đầu vào cho các thuật toán phân lớp.

In [46]:
# Import thư viện 
!pip install scikit-learn
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, classification_report

# Các mô hình phân lớp
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Load dataset Digits 
digits = load_digits()
X = digits.data          # dữ liệu đặc trưng
y = digits.target        # nhãn lớp
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Number of classes:", len(np.unique(y)))


Shape of X: (1797, 64)
Shape of y: (1797,)
Number of classes: 10



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Chia dữ liệu

Theo yêu cầu của đề bài, tập dữ liệu được chia ngẫu nhiên thành ba phần với tỷ lệ 60% (Train) - 20% (Validation) - 20% (Test). Việc chia làm hai bước được thực hiện với hạt giống ngẫu nhiên cố định (random_state=42) để đảm bảo kết quả có thể tái lập được và tỷ lệ các lớp được giữ nguyên (stratify).

Sau đó, dữ liệu sẽ được chuẩn hóa (Standard Scaling). Bước tiền xử lý này rất quan trọng vì nó giúp đưa các đặc trưng về cùng một thang đo, từ đó giúp các mô hình dựa trên khoảng cách (KNN, ANN) và các mô hình dựa trên Gradient Descent (Logistic Regression) hội tụ hiệu quả và nhanh chóng hơn.

In [47]:
# Chia dữ liệu 
# Tách train (60%) và temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Tách tiếp temp thành validation (20%) và test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)


Train shape: (1078, 64) (1078,)
Validation shape: (359, 64) (359,)
Test shape: (360, 64) (360,)


# 3. Hồi quy Logistic (Logistic Regression)

Trước khi huấn luyện mô hình Logistic Regression, ta tiến hành chuẩn hóa dữ liệu (Standard Scaling) để tối ưu hóa quá trình hội tụ của thuật toán. Sau đó, ta thực hiện tìm kiếm siêu tham số tối ưu bằng cách đánh giá các giá trị khác nhau của tham số điều chuẩn C (Inverse of regularization strength) trên tập Validation. Cuối cùng, mô hình với tham số tốt nhất sẽ được đánh giá độ chính xác lần cuối trên tập Test để xác định hiệu năng khách quan của mô hình.

In [48]:


# --- Chuẩn hóa dữ liệu ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# --- Logistic Regression ---
# Thử nhiều giá trị C (regularization)
C_values = [0.01, 0.1, 1, 10, 100]
best_acc = 0
best_C = None
for C in C_values:
    clf = LogisticRegression(C=C, max_iter=1000, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_val_pred = clf.predict(X_val_scaled)
    acc = accuracy_score(y_val, y_val_pred)
    if acc > best_acc:
        best_acc = acc
        best_C = C
        best_model = clf

print(f"Best C: {best_C}, Validation Accuracy: {best_acc:.4f}")
# Lưu model và params riêng cho Logistic Regression 
best_lr_model = best_model
best_lr_params = {'C': best_C}

# Đánh giá trên test set 
y_test_pred = best_model.predict(X_test_scaled)
test_acc = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred, average=None)
print(f"Test Accuracy: {test_acc:.4f}")
print("Recall per class:", test_recall)


Best C: 100, Validation Accuracy: 0.9694
Test Accuracy: 0.9556
Recall per class: [0.97142857 0.91891892 0.94285714 1.         0.94444444 1.
 0.97222222 0.97222222 0.94285714 0.88888889]


# 4. Cây Quyết định (Decision Tree)

Mô hình Cây Quyết định là một thuật toán phi tham số (non-parametric), nên không yêu cầu chuẩn hóa dữ liệu (Scaling). Tuy nhiên, mô hình này rất dễ bị học vẹt (Overfitting) trên dữ liệu huấn luyện. Do đó, mục tiêu chính của việc tối ưu siêu tham số là kiểm soát độ phức tạp của cây bằng cách tìm kiếm tổ hợp tốt nhất cho độ sâu tối đa (max_depth) và số lượng mẫu tối thiểu để tách nút (min_samples_split). Các tham số này sẽ được đánh giá trên tập Validation.

In [49]:
# --- Decision Tree ---
from sklearn.tree import DecisionTreeClassifier

# Các giá trị thử nghiệm
max_depth_values = [None, 5, 10, 15, 20]
min_samples_split_values = [2, 5, 10]

best_acc = 0
best_params = {}
for max_depth in max_depth_values:
    for min_samples_split in min_samples_split_values:
        clf = DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42
        )
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, y_val_pred)
        if acc > best_acc:
            best_acc = acc
            best_model = clf
            best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split}

print(f"Best params: {best_params}, Validation Accuracy: {best_acc:.4f}")
# Lưu model và params riêng cho Decision Tree 
best_dt_model = best_model
best_dt_params = best_params


# --- Đánh giá trên test set ---
y_test_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred, average=None)
print(f"Test Accuracy: {test_acc:.4f}")
print("Recall per class:", test_recall)


Best params: {'max_depth': None, 'min_samples_split': 2}, Validation Accuracy: 0.8524
Test Accuracy: 0.8111
Recall per class: [0.85714286 0.67567568 0.82857143 0.75675676 0.86111111 1.
 0.94444444 0.80555556 0.68571429 0.69444444]


# 5. K-Láng giềng gần nhất (K-Nearest Neighbors - KNN)

KNN là một thuật toán dựa trên khoảng cách, vì vậy, chuẩn hóa dữ liệu (Scaling) là bước bắt buộc để đảm bảo hiệu suất tối ưu. Ta sẽ tiến hành tìm kiếm siêu tham số tối ưu bằng cách đánh giá các giá trị khác nhau của số lượng láng giềng K (n_neighbors) trên tập Validation. Mô hình với giá trị $K$ tốt nhất sẽ được lưu lại để đánh giá cuối cùng trên tập Test.

In [50]:
#  K-Nearest Neighbors 
from sklearn.neighbors import KNeighborsClassifier

# Chuẩn hóa dữ liệu cho KNN
scaler_knn = StandardScaler()
X_train_scaled = scaler_knn.fit_transform(X_train)
X_val_scaled = scaler_knn.transform(X_val)
X_test_scaled = scaler_knn.transform(X_test)

# Thử nhiều giá trị K
k_values = [1, 3, 5, 7, 9, 11, 13]
best_acc = 0
best_k = None

for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_scaled, y_train)
    y_val_pred = clf.predict(X_val_scaled)
    acc = accuracy_score(y_val, y_val_pred)
    if acc > best_acc:
        best_acc = acc
        best_k = k
        best_model = clf

print(f"Best K: {best_k}, Validation Accuracy: {best_acc:.4f}")
# --- Lưu model và params riêng cho KNN ---
best_knn_model = best_model
best_knn_params = {'K': best_k}


# --- Đánh giá trên test set ---
y_test_pred = best_model.predict(X_test_scaled)
test_acc = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred, average=None)
print(f"Test Accuracy: {test_acc:.4f}")
print("Recall per class:", test_recall)


Best K: 1, Validation Accuracy: 0.9721
Test Accuracy: 0.9750
Recall per class: [1.         0.97297297 0.94285714 1.         0.94444444 1.
 1.         1.         0.88571429 1.        ]


# 6. Mạng Nơ-ron Nhân tạo (Artificial Neural Network - ANN)

ANN là một trong những mô hình mạnh mẽ nhất, nhưng cũng yêu cầu chuẩn hóa dữ liệu để đảm bảo quá trình lan truyền ngược (Backpropagation) hội tụ nhanh chóng và hiệu quả. Ta tiến hành tối ưu hai siêu tham số quan trọng: cấu trúc lớp ẩn (hidden_layer_sizes) và hàm kích hoạt (activation). Mô hình có hiệu suất tốt nhất trên tập Validation sẽ được sử dụng để đánh giá cuối cùng trên tập Test.

In [51]:
# --- Artificial Neural Network (MLPClassifier) ---
from sklearn.neural_network import MLPClassifier

# Chuẩn hóa dữ liệu cho ANN
scaler_ann = StandardScaler()
X_train_scaled = scaler_ann.fit_transform(X_train)
X_val_scaled = scaler_ann.transform(X_val)
X_test_scaled = scaler_ann.transform(X_test)

# Các giá trị thử nghiệm
hidden_layer_sizes_list = [(50,), (100,), (50,50)]
activation_list = ['relu', 'tanh']

best_acc = 0
best_params = {}
for hidden_layer_sizes in hidden_layer_sizes_list:
    for activation in activation_list:
        clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                            activation=activation,
                            max_iter=500,
                            random_state=42)
        clf.fit(X_train_scaled, y_train)
        y_val_pred = clf.predict(X_val_scaled)
        acc = accuracy_score(y_val, y_val_pred)
        if acc > best_acc:
            best_acc = acc
            best_model = clf
            best_params = {'hidden_layer_sizes': hidden_layer_sizes,
                           'activation': activation}

print(f"Best params: {best_params}, Validation Accuracy: {best_acc:.4f}")
# --- Lưu model và params riêng cho ANN ---
best_ann_model = best_model
best_ann_params = best_params



# --- Đánh giá trên test set ---
y_test_pred = best_model.predict(X_test_scaled)
test_acc = accuracy_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred, average=None)
print(f"Test Accuracy: {test_acc:.4f}")
print("Recall per class:", test_recall)


Best params: {'hidden_layer_sizes': (50, 50), 'activation': 'relu'}, Validation Accuracy: 0.9833
Test Accuracy: 0.9722
Recall per class: [0.97142857 0.97297297 1.         1.         0.97222222 1.
 1.         1.         0.94285714 0.86111111]


# 7. Bảng tổng hợp kết quả, So sánh Hiệu năng và Kết luận

In [56]:
# --- 7. Tổng hợp & Phân tích Kết quả Cuối cùng ---

results = []

# 1. Logistic Regression
results.append({
    'Model': 'Logistic Regression',
    'Best Params': str(best_lr_params), # Chuyển thành string cho gọn
    'Test Accuracy': accuracy_score(y_test, best_lr_model.predict(X_test_scaled)),
    'Recall per class': recall_score(y_test, best_lr_model.predict(X_test_scaled), average=None)
})

# 2. Decision Tree
results.append({
    'Model': 'Decision Tree',
    'Best Params': str(best_dt_params),
    'Test Accuracy': accuracy_score(y_test, best_dt_model.predict(X_test)),
    'Recall per class': recall_score(y_test, best_dt_model.predict(X_test), average=None)
})

# 3. KNN
results.append({
    'Model': 'KNN',
    'Best Params': str(best_knn_params),
    'Test Accuracy': accuracy_score(y_test, best_knn_model.predict(X_test_scaled)),
    'Recall per class': recall_score(y_test, best_knn_model.predict(X_test_scaled), average=None)
})

# 4. ANN
results.append({
    'Model': 'ANN',
    'Best Params': str(best_ann_params),
    'Test Accuracy': accuracy_score(y_test, best_ann_model.predict(X_test_scaled)),
    'Recall per class': recall_score(y_test, best_ann_model.predict(X_test_scaled), average=None)
})

# --- Xử lý DataFrame để tách cột Recall ---

# B1: Tạo DF ban đầu
temp_df = pd.DataFrame(results)

# B2: Tách cột 'Recall per class' (đang là list) thành 10 cột riêng biệt
recall_df = pd.DataFrame(temp_df['Recall per class'].tolist(), 
                         columns=[f'Recall_Class_{i}' for i in range(10)])

# B3: Ghép lại (Bỏ cột list cũ, thêm 10 cột mới)
results_df = pd.concat([temp_df.drop('Recall per class', axis=1), recall_df], axis=1)

# --- Hiển thị ---
pd.set_option('display.max_columns', None) # Hiện tất cả các cột
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)      # Lấy 4 chữ số thập phân

results_df

,Model,Best Params,Test Accuracy,Recall_Class_0,Recall_Class_1,Recall_Class_2,Recall_Class_3,Recall_Class_4,Recall_Class_5,Recall_Class_6,Recall_Class_7,Recall_Class_8,Recall_Class_9
0,Logistic Regression,{'C': 100},0.9556,0.9714,0.9189,0.9429,1.0000,0.9444,1.0,0.9722,0.9722,0.9429,0.8889
1,Decision Tree,"{'max_depth': None, 'min_samples_split': 2}",0.8111,0.8571,0.6757,0.8286,0.7568,0.8611,1.0,0.9444,0.8056,0.6857,0.6944
2,KNN,{'K': 1},0.9750,1.0000,0.9730,0.9429,1.0000,0.9444,1.0,1.0000,1.0000,0.8857,1.0000
3,ANN,"{'hidden_layer_sizes': (50, 50), 'activation':...",0.9722,0.9714,0.9730,1.0000,1.0000,0.9722,1.0,1.0000,1.0000,0.9429,0.8611



## Dựa trên bảng tổng hợp kết quả (Accuracy và Recall theo từng lớp) trên tập kiểm tra (Test Set), ta có những phân tích chi tiết về hiệu năng của các mô hình như sau:

## 1. Đánh giá Hiệu năng Tổng thể

* **Nhóm hiệu suất cao (KNN & ANN):**
    * **K-Nearest Neighbors (KNN)** đạt độ chính xác cao nhất (**97.50%**). Điều này cho thấy tập dữ liệu *Digits* có tính chất phân cụm rõ ràng trong không gian đặc trưng. Các chữ số cùng loại nằm gần nhau, giúp thuật toán dựa trên khoảng cách như KNN hoạt động cực kỳ hiệu quả.
    * **Artificial Neural Network (ANN)** theo sát với **97.22%**. Mạng nơ-ron với cấu trúc 2 lớp ẩn đã học tốt các đặc trưng phi tuyến tính phức tạp của ảnh chữ số.

* **Mô hình tuyến tính (Logistic Regression):**
    * Đạt **95.56%**, một kết quả rất ấn tượng. Điều này chứng tỏ rằng dù là dữ liệu ảnh, nhưng sau khi được chuẩn hóa (*StandardScaler*), ranh giới giữa các lớp chữ số tương đối rõ ràng và có thể phân tách tốt bằng các siêu phẳng tuyến tính.

* **Mô hình hiệu suất thấp (Decision Tree):**
    * Chỉ đạt **81.11%**, thấp hơn đáng kể so với các mô hình còn lại.

## 2. Phân tích Chi tiết theo Lớp (Class-level Analysis)

Quan sát cột *Recall per class*, ta thấy rõ sự khác biệt về độ ổn định giữa các mô hình:

* **Điểm yếu của Decision Tree:**
    * Mô hình này gặp khó khăn lớn trong việc nhận diện các chữ số có hình dáng mảnh hoặc dễ gây nhầm lẫn. Cụ thể, độ nhạy (Recall) cực thấp ở các lớp: **số 1 (67.6%)**, **số 8 (68.6%)**, và **số 9 (69.4%)**.
    * *Nguyên nhân:* Do Decision Tree phân chia không gian dựa trên các đường cắt vuông góc trục (orthogonal splits), nó khó nắm bắt được các đường cong hoặc nét xiên biến thiên của chữ viết tay, dẫn đến hiện tượng *overfitting* cục bộ nhưng kém tổng quát hóa.

* **Sự vượt trội của KNN:**
    * KNN đạt độ nhạy tuyệt đối **100%** ở hầu hết các lớp (0, 3, 4, 5, 6, 8), cho thấy mô hình này bao phủ rất tốt các biến thể của chữ viết tay trong tập dữ liệu này.

## 3. Kết luận

1.  **Chuẩn hóa dữ liệu là tiên quyết:** Việc KNN và ANN vượt trội khẳng định tầm quan trọng của bước *StandardScaler*, đặc biệt với các thuật toán dựa trên khoảng cách và tối ưu gradient.
2.  **Lựa chọn mô hình:** Với tập dữ liệu nhỏ và số chiều trung bình (64 features) như Digits, **KNN** là lựa chọn tốt nhất về độ chính xác. Tuy nhiên, nếu xét về tốc độ dự đoán (inference time) và khả năng triển khai thực tế, **ANN** hoặc **Logistic Regression** sẽ là những ứng viên cân bằng hơn.